**Extraer en un archivo CSV, la ciudad, dirección y si hay o no una farmacia en las sucursales de Hipermaxi**

In [2]:
import requests
respuesta = requests.get("http://localhost:8000/sucursales-hipermaxi.json")
data = respuesta.json()
data

[{'id': '44',
  'title': 'FARMACIA BAJIO',
  'description': '',
  'street': 'AV. DOBLE VIA A LA GUARDIA KM. 6 S/N ZONA EL BAJIO',
  'city': 'Santa Cruz',
  'state': '',
  'postal_code': '',
  'country': 'Bolivia',
  'lat': '-17.827811',
  'lng': '-63.226611',
  'phone': '',
  'fax': '',
  'email': '',
  'website': '',
  'logo_id': '0',
  'path': None,
  'marker_id': '153',
  'description_2': '',
  'open_hours': '{"mon":"1","tue":"1","wed":"1","thu":"1","fri":"1","sat":"1","sun":"1"}',
  'ordr': '0',
  'brand': None,
  'product': None,
  'slug': 'farmacia-bajio-santa-cruz',
  'categories': '18',
  'days_str': 'Lun, Mar, Mie, Jue, Vie, Sab, Dom'},
 {'id': '45',
  'title': 'FARMACIA BARRON',
  'description': '',
  'street': 'CALLE BARRON S/N BARRIO CENTRAL',
  'city': 'Santa Cruz',
  'state': '',
  'postal_code': '',
  'country': 'Bolivia',
  'lat': '-17.7757',
  'lng': '-63.175125',
  'phone': '',
  'fax': '',
  'email': '',
  'website': '',
  'logo_id': '0',
  'path': None,
  'marker_id

In [5]:
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse

cities = [match.value for match in parse('$[*].city').find(data)]

In [8]:
addresses = [match.value for match in parse('$[*].street').find(data)]

In [11]:
def check_pharmacy(slug):
    return 'tiene farmacia' if slug.startswith('farmacia') else 'no tiene farmacia'

with_pharmacy = [check_pharmacy(match.value) for match in parse('$[*].slug').find(data)]

In [15]:
hipermaxi_tuples = list(zip(cities, addresses, with_pharmacy))
hipermaxi_records = [{"city":t[0], "address": t[1], "has_pharmacy": t[2]} for t in hipermaxi_tuples]

In [17]:
import csv

with open("hipermaxi.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=['city', 'address', 'has_pharmacy'])
    writer.writeheader()
    writer.writerows(hipermaxi_records)

**Extraer los siguientes datos de los Metales de transición en un archivo CSV: Nombre, número atómico, símbolo**

In [20]:
#https://es.wikipedia.org/wiki/Tabla_peri%C3%B3dica_de_los_elementos
import requests
respuesta = requests.get("http://localhost:8000/Tabla_peri%C3%B3dica_de_los_elementos.html")
# respuesta.text

In [23]:
target_color = '#81F7BE'
from bs4 import BeautifulSoup
pagina = BeautifulSoup(respuesta.text, "html.parser" )
pagina

In [28]:
element_tds = pagina.body.find_all(name="td", attrs={"width":"5%", "bgcolor": target_color})
element_tds[0]

<td align="center" bgcolor="#81F7BE" width="5%"><b>21</b><br/><a href="https://es.wikipedia.org/wiki/Escandio" title="Escandio">Sc</a>
</td>

In [29]:
def extract_element_data(td):
    atomic_weight = td.find(name="b").text
    element_link = td.find(name="a")
    name = element_link.text
    symbol = element_link['title']
    return {'atomic_weight': atomic_weight, "name": name, "symbol": symbol }

In [31]:
extracted_metals = [extract_element_data(td) for td in element_tds]

In [32]:
import csv

with open("metals.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=['atomic_weight', 'name', 'symbol'])
    writer.writeheader()
    writer.writerows(extracted_metals)

**Extraer en un archivo CSV los siguientes datos de las noticias del autor 'Swati Dubey': fecha, hora, título y url**

In [3]:
#https://inshorts.deta.dev/news?category=science
import requests
respuesta = requests.get("http://localhost:8000/inshort-news.json")
data = respuesta.json()
data

{'category': 'science',
 'data': [{'author': 'Apaar Sharma',
   'content': 'A 36-year-old man in Italy has tested positive for COVID-19, monkeypox, and HIV at the same time after returning from a trip in Spain where he engaged in unprotected sex, according to researchers from the University of Catania. This is the first known case of a person testing positive for the three viruses at the same time.',
   'date': '25 Aug 2022,Thursday',
   'id': '10e352f0286e432e89ed5c9aa4c4711b',
   'imageUrl': 'https://static.inshorts.com/inshorts/images/v1/variants/jpg/m/2022/08_aug/25_thu/img_1661416333757_164.jpg?',
   'readMoreUrl': 'https://www.independent.co.uk/news/health/monkeypox-covid-hiv-signs-patient-italy-b2152204.html?utm_campaign=fullarticle&utm_medium=referral&utm_source=inshorts ',
   'time': '02:47 pm',
   'title': 'Italian man tests positive for COVID-19, monkeypox & HIV at the same time',
   'url': 'https://www.inshorts.com/en/news/italian-man-tests-positive-for-covid19-monkeypox-hi